# Laboratorio 2022-2023

##  Sesión 18: Cadenas de Markov finitas en tiempo discreto

Queremos simular la evolución de sistemas descritos por cadenas de Markov que involucran un número finito de estados, en tiempo discreto. Para ello tendremos que ser capaces de, desde un estado $j$,  *sortear* a qué estado llegaremos en el paso siguiente, de acuerdo con las probabilidades dadas por la columna $j$ de la matriz $P$. El siguiente ejercicio nos va a ayudar a ello.


**Ejercicio 1.-** De un cine, con tres salidas, $1/3$ de los espectadores sale por la salida $0$, $1/5$ por la $1$ y el resto, $7/15$, por la $2$. Se elige, al azar, un espectador, ¿por qué puerta saldrá? Esta situación se puede simular decidiendo conforme a la siguiente regla:
- se divide el intervalo $[0,1)$ en tres subintervalos de longitudes $1/3$, $1/5$ y $7/15$ respectivamente, es decir:
$$[0,1)= I_0\cup I_1\cup I_2=[0,\tfrac13)\cup [\tfrac 13,\tfrac13+\tfrac15) \cup [\tfrac13+\tfrac15,1)$$
- se decide la puerta de salida por el subintervalo en que se sitúe el número $t={\tt random}()$: si $t\in I_0$, la salida $0$; si $t\in I_1$, la $1$; por la $2$ en otro caso.

Es claro que de esta manera, se decidirá cada puerta de salida conforme a las probabilidades que nos han referido. La misma estrategia es válida para cualquier **vector** de probabilidades.

Codifica un programa *paso(v)* que, dado un vector $v$ de probabilidad (es decir, con suma de sus coordenadas igual a 1), genere de manera aleatoria un entero $k$ entre $0$ y *len(v)-1*, teniendo cada uno de estos números una probabilidad *$v[k]$*.

*Sugerencia.-* Construye un vector $F$ con elementos $F[k]=\sum_{j=0}^k v[j]$. Genera un número aleatorio $a$ entre 0 y 1. La salida será el primer $k$ tal que $a<F[k]$.

In [23]:
# Primera versión, siguiendo la sugerencia

def paso(v):
    F=[v[0]]
    for k in xsrange(1,len(v)):
        F.append(F[k-1]+v[k])
    a=random()
    indice=0
    while a> F[indice]:
        indice+=1
    return indice 

In [24]:
#Pruebo si funciona bien, con las probabilidades de las tres salidas del cine
v=[1/3, 1/5, 7/15]

t0=walltime()
ceros=unos=doses=0 #Cadena de asignaciones para inicializar a 0 los tres contadores
rep=10^6
for _ in xsrange(rep):
    actual=paso(v)
    if actual==0: ceros+=1
    elif actual==1: unos+=1
    else: doses+=1
print('Tarda',walltime(t0),'segundos')
print ((ceros/rep).n(),(unos/rep).n(), (doses/rep).n())
print((1/3).n(), (1/5).n(),(7/15).n()) 

Tarda 14.849177122116089 segundos
0.333816000000000 0.200027000000000 0.466157000000000
0.333333333333333 0.200000000000000 0.466666666666667


In [25]:
# Segunda versión, que esperamos sea más eficiente. Primero genero el número aleatorio a, y luego voy calculando valores de F[k] 
# hasta llegue a un k tal que a<F[k]. Ahí paro.

def paso(v): 
    a=random()
    Fk=v[0]
    k=0
    while a>Fk:
        Fk+=v[k+1]
        k+=1
    return k

In [26]:
# Probamos si funciona bien, con las probabilidades de las tres salidas del cine
v=[1/3, 1/5, 7/15]

t0=walltime()
ceros=unos=doses=0 #Cadena de asignaciones para inicializar a 0 los tres contadores
rep=10^6
for _ in xsrange(rep):
    actual=paso(v)
    if actual==0:
        ceros+=1
    elif actual==1:
        unos+=1
    else:
        doses+=1
print('Tarda',walltime(t0),'segundos')
print ((ceros/rep).n(),(unos/rep).n(), (doses/rep).n())
print((1/3).n(), (1/5).n(),(7/15).n()) 

Tarda 4.1134350299835205 segundos
0.333317000000000 0.199964000000000 0.466719000000000
0.333333333333333 0.200000000000000 0.466666666666667


In [27]:
def pasoF(F):
    tirada=random()
    indice=0
    while tirada> F[indice]:
        indice+=1
    return indice 

In [28]:
# Probamos si funciona bien, con las probabilidades de las tres salidas del cine

F=[1/3,1/3+1/5,1]  # Vector de distribución
ceros=unos=doses=0 #Cadena de asignaciones para inicializar a 0 los tres contadores

t0=walltime()
rep=10^6
for _ in xsrange(rep):
    actual=pasoF(F)
    if actual==0:
        ceros+=1
    elif actual==1:
        unos+=1
    else:
        doses+=1
print('Tarda',walltime(t0),'segundos')
print ((ceros/rep).n(),(unos/rep).n(), (doses/rep).n())
print((1/3).n(), (1/5).n(),(7/15).n()) 

Tarda 2.914644479751587 segundos
0.333199000000000 0.200077000000000 0.466724000000000
0.333333333333333 0.200000000000000 0.466666666666667


____________

**Ejercicio 2.-** Supongamos que una cadena de Markov con estados $0,1,2$ tiene matriz de transición $P$ descrita por la siguiente tabla:

$$\begin{array}{|c|c|c|c|} \hline & 0 & 1 & 2\\ \hline 0 & 0.1 & 0.9 & 0.1 \\ 1 & 0.2 & 0.1 & 0.8\\ 2 & 0.7 & 0 & 0.1 \\ \hline \end{array}$$


Sea $X_n$ *el estado de la cadena en el tiempo $n$.* 


**a)** Simula la evolución del sistema durante $1000$ pasos suponiendo que incialmente está en el estado $1$. Contabiliza, por medio de un diccionario, el número de veces que el sistema ha estado en cada estado (*sugerencia.-* el método *.column(j)* extrae la j-ésima columna de una matriz).  Compara las proporciones del tiempo total que el sistema pasó en cada estado, que  has obtenido, con el autovector de la matriz $P$ (regular) correspondiente al autovalor 1, normalizado de forma que sea un vector de probabilidad. 

In [30]:
P=matrix(QQ, 3,[0.1,0.9,0.1,0.2,0.1,0.8,0.7,0,0.1])
show(P)

[1/10 9/10 1/10]
[ 1/5 1/10  4/5]
[7/10    0 1/10]

In [31]:
estado=1
frec=dict([(1,1)])
rep=10^5
for _ in xsrange(rep):
    estado=paso(P.column(estado))
    if estado in frec: frec[estado]+=1
    else: frec[estado]=1
prop=[(frec[estado]/rep).n(digits=4) for estado in [0..2] ]
prop

[0.3725, 0.3388, 0.2887]

In [32]:
show(P^2)

[ 13/50   9/50  37/50]
[   3/5 19/100   9/50]
[  7/50 63/100   2/25]

In [33]:
autovdom=(P-identity_matrix(3)).right_kernel().basis()[0] # Generador del núcleo, por la derecha, de P-I
autovdom=autovdom/sum(autovdom) # Normalizado
show(autovdom.n(digits=4))

(0.3716, 0.3395, 0.2890)

**b)** Simula la evolución del sistema durante $1000$ pasos si se parte de un estado inicial elegido aleatoriamente según la distribución de probabilidades  siguiente: $P(X_0=0)=0.3$, $P(X_0=1)=0.4$ y $P(X_0=2)=0.3$. Contabiliza las veces que el sistema está en cada estado. ¿Ves alguna diferencia significativa con el caso anterior, en el que se partía de un estado determinado? *Sugerencia.-* Para el "sorteo" del estado inicial puedes usar también la funcion $paso$ del Ejercicio 1.

In [34]:
estado=paso([0.3,0.4,0.3])
frec=dict([(estado,1)])
rep=10^5
for _ in xsrange(rep):
    estado=paso(P.column(estado))
    if estado in frec: frec[estado]+=1
    else: frec[estado]=1
prop=[(frec[estado]/rep).n(digits=4) for estado in [0..2] ]
prop

[0.3713, 0.3390, 0.2897]

**c)** Estima experimentalmente la probabilidad de, partiendo del estado $0$, acabar en $5$ pasos en el estado $2$. Compara con el elemento $(2,0)$ de la matriz $P^5$.

In [35]:
rep=10^5
exitos=0
for _ in xsrange(rep):
    estado=0
    for _ in xsrange(5): estado=paso(P.column(estado))
    if estado==2: exitos+=1
(exitos/rep).n(digits=4), (P^5)[2,0].n(digits=4)

(0.2371, 0.2380)

**d)** Estima experimentalmente la probabilidad de que el sistema empiece en el estado 0 y tras cinco pasos se encuentre en el estado 2, suponiendo que el estado inicial se elige aleatoriamente como en el apartado **b)**. Compara el valor obtenido con el producto de $P(X_0=0)$ con el elemento $(2,0)$ de la matriz $P^5$.

In [36]:
rep=10^5
exitos=0
for _ in xsrange(rep):
    estado=paso([0.3,0.4,0.3])
    if estado==0:
        for _ in xsrange(5): estado=paso(P.column(estado))
        if estado==2: exitos+=1
(exitos/rep).n(digits=4), 0.3*(P^5)[2,0].n(digits=4)

(0.07133, 0.07140)

_______________

**Ejercicio 3.-**  Introducimos un ratón en el siguiente laberinto:
<center>
    <img src="laberinto.png" alt="Laberinto" width="26%" />
</center>

En la habitación $8$ hay un agujero en la pared: si el ratón llega a ella escapa y vive una vida feliz en el exterior. En la habitación $7$ hay un gato y... bueno, no demos detalles, pero el caso es que las habitaciones $7$ y $8$ se consideran estados absorbentes. Se supone, además, que el ratón no aprende por dónde ha pasado, de manera que se mueve por el recinto aleatoriamente (elige, con igual probabilidad, la puerta de salida de la habitación entre las posibles).

**a)** Estima experimentalmente la probabilidad de que el ratón pueda escapar suponiendo que empieza en la casilla $2$.


In [14]:
P=matrix(QQ,9) #Esto es una forma de generar una matriz de ceros
#Dos puertas de salida
P[1,0]=P[2,0]=P[4,6]=P[5,6]=1/2
#Tres puertas de salida
P[0,1]=P[3,1]=P[7,1]=P[0,2]=P[3,2]=P[8,2]=P[3,4]=P[6,4]=P[7,4]=P[3,5]=P[6,5]=P[8,5]=1/3
#Cuatro puertas de salida
P[1,3]=P[2,3]=P[4,3]=P[5,3]=1/4
#Absorbentes
P[7,7]=P[8,8]=1
show(P)

[  0 1/3 1/3   0   0   0   0   0   0]
[1/2   0   0 1/4   0   0   0   0   0]
[1/2   0   0 1/4   0   0   0   0   0]
[  0 1/3 1/3   0 1/3 1/3   0   0   0]
[  0   0   0 1/4   0   0 1/2   0   0]
[  0   0   0 1/4   0   0 1/2   0   0]
[  0   0   0   0 1/3 1/3   0   0   0]
[  0 1/3   0   0 1/3   0   0   1   0]
[  0   0 1/3   0   0 1/3   0   0   1]

In [15]:
# Esto no se pregunta, pero podemos simular recorridos del ratón saliendo desde 2

for _ in xsrange(10):
    estados=[]
    estado=2
    estados.append(estado)
    while estado!=7 and estado!=8:
        estado=paso(P.column(estado))
        estados.append(estado)
    print(estados)

[2, 0, 2, 8]
[2, 0, 2, 8]
[2, 3, 5, 8]
[2, 0, 2, 0, 2, 8]
[2, 8]
[2, 3, 5, 8]
[2, 3, 1, 7]
[2, 8]
[2, 3, 2, 0, 2, 8]
[2, 3, 5, 3, 4, 6, 5, 6, 5, 6, 4, 7]


In [16]:
# Probabilidad de escapar empezando en 2 

rep=10^5
exitos=0
for _ in xsrange(rep):
    estado=2
    while estado!=7 and estado!=8: estado=paso(P.column(estado))
    if estado==8: exitos+=1
(exitos/rep).n()

0.665720000000000

**b)** Estima experimentalmente la probabilidad de que el ratón sea cazado por el gato suponiendo que ha sido introducido al azar en el laberinto.


In [17]:
#Encontrarse con el gato si empieza al azar (con misma probabilidad de empezar en cualquier sitio)

rep=10^5
exitos=0
for _ in xsrange(rep):
    estado=randint(0,8)
    while estado!=7 and estado!=8: estado=paso(P.column(estado))
    if estado==7: exitos+=1
(exitos/rep).n()

0.499910000000000

**c)** Estima experimentalmente el tiempo medio que un ratón que empieza en $6$ y no es cazado por el gato emplea hasta que consigue escapar.


In [18]:
# Tiempo medio de 6 a agujero de salida, suponiendo que llega

rep=10^5
tiempototal=0
exitos=0
for _ in xsrange(rep):
    tiempo=0
    estado=6
    while estado!=7 and estado!=8:
        tiempo+=1
        estado=paso(P.column(estado))
    if estado==8:
        exitos+=1
        tiempototal+=tiempo
(tiempototal/exitos).n()

5.97766978593518

_______________

**Ejercicio 4.-** Ladan Afar, tenista famoso por su poderoso servicio pero un poco frágil mentalmente, te ha contratado para un estudio matemático. Analizando las estadísticas de sus partidos determinas, en los juegos en los que Ladan está al servicio, la frecuencia con la que Ladan gana el siguiente  punto en función de cuál sea el marcador en ese momento. La figura siguiente resume dichas frecuencias:
<center>
    <img src="tenis.png" alt="Tenis" width="60%" />
</center>

Nota: no se ha considerado que _30-30_ e _iguales_ sean dos resultados distintos, puesto que son en cierto sentido equivalentes. Lo mismo ocurre con los marcadores de _40-30_ y _ventaja al servicio_ (V+) ó con _30-40_ y _ventaja al resto_ (V-).

**a)** Estima qué probabilidad tiene Ladan Afar de ganar un juego en el que está al servicio, y cuántas veces, en promedio, pasa uno de esos juegos por el marcador "Iguales".

In [37]:
#Mi forma de numerar los estados: 
#['0-0','15-0','0-15','30-0','15-15','0-30','40-0','30-15','15-30','0-40','40-15','Igu','15-40','V+','V-','Gana','Pierde']

P=matrix(RDF,17) #Matriz de ceros
P[1,0], P[2,0]=0.6,0.4
P[3,1], P[4,1]=0.6,0.4
P[4,2], P[5,2]=0.6,0.4
P[6,3], P[7,3]=0.6,0.4
P[7,4], P[8,4]=0.6,0.4
P[8,5], P[9,5]=0.4,0.6
P[10,6], P[15,6]=0.4,0.6
P[10,7], P[11,7]=0.6,0.4
P[11,8], P[12,8]=0.4,0.6
P[12,9], P[16,9]=0.2,0.8
P[13,10], P[15,10]=0.4,0.6
P[13,11], P[14,11]=0.5,0.5
P[14,12], P[16,12]=0.3,0.7
P[11,13], P[15,13]=0.4,0.6
P[11,14], P[16,14]=0.3,0.7
P[15,15]=1
P[16,16]=1
show(P)

[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.6 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.4 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.6 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.4 0.6 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.4 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.6 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.4 0.6 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.4 0.4 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.6 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.4 0.6 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.4 0.4 0.0 0.0 0.0 0.0 0.4 0.3 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.6 0.2 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.4 0.5 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.5 0.3 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.6 0.0 0.0 0.0 0.6 0.0 0.0 0.6 0.0 1.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.8 0.0 0.0 0.7 0.0 0.7 0.0 1.0]

In [20]:
rep=10^5
exitos=0
iguales=0
for _ in xsrange(rep):
    estado=0
    while estado!=15 and estado!=16:
        estado=paso(P.column(estado))
        if estado==11: iguales+=1
    if estado==15: exitos+=1
print('La probabilidad de ganar es', (exitos/rep).n(digits=3))
print('En promedio, se pasa por "iguales"', (iguales/rep).n(digits=3), 'veces')

La probabilidad de ganar es 0.581
En promedio, se pasa por "iguales" 0.527 veces


**b)** ¿Cuál sería la probabilidad que tiene Ladan Afar de ganar un juego en el que está al servicio si fuera más duro mentalmente y su probabilidad de ganar cualquier punto fuese siempre de $0.6$, independientemente del resultado que figurase en el marcador en ese momento?

In [38]:
P=matrix(RDF,17) #Matriz de ceros
P[1,0], P[2,0]=0.6,0.4
P[3,1], P[4,1]=0.6,0.4
P[4,2], P[5,2]=0.6,0.4
P[6,3], P[7,3]=0.6,0.4
P[7,4], P[8,4]=0.6,0.4
P[8,5], P[9,5]=0.6,0.4
P[10,6], P[15,6]=0.4,0.6
P[10,7], P[11,7]=0.6,0.4
P[11,8], P[12,8]=0.6,0.4
P[12,9], P[16,9]=0.6,0.4
P[13,10], P[15,10]=0.4,0.6
P[13,11], P[14,11]=0.6,0.4
P[14,12], P[16,12]=0.6,0.4
P[11,13], P[15,13]=0.4,0.6
P[11,14], P[16,14]=0.6,0.4
P[15,15]=1
P[16,16]=1
show(P)

[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.6 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.4 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.6 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.4 0.6 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.4 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.6 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.4 0.6 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.4 0.6 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.4 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.4 0.6 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.4 0.6 0.0 0.0 0.0 0.0 0.4 0.6 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.4 0.6 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.4 0.6 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.4 0.6 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.6 0.0 0.0 0.0 0.6 0.0 0.0 0.6 0.0 1.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.4 0.0 0.0 0.4 0.0 0.4 0.0 1.0]

In [39]:
rep=10^5
exitos=0
iguales=0
for _ in xsrange(rep):
    estado=0
    while estado!=15 and estado!=16:
        estado=paso(P.column(estado))
        if estado==11: iguales+=1
    if estado==15: exitos+=1
print('La probabilidad de ganar es ahora', (exitos/rep).n(digits=3))

La probabilidad de ganar es ahora 0.738
